### Scrape pro-football-reference.com for total stats in 2021

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
year = 2021
url = "https://www.pro-football-reference.com/years/{}/fantasy.htm#".format(year)
html = urlopen(url)
soup = BeautifulSoup(html)

In [3]:
headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
headers = headers[1:]

In [4]:
rows = soup.findAll('tr', class_ = lambda table_rows: table_rows != "thead")
player_stats = [[td.getText() for td in rows[i].findAll('td')]
               for i in range(len(rows))]
player_stats = player_stats[2:]

In [5]:
stats = pd.DataFrame(player_stats, columns = headers)

In [6]:
stats = stats.replace(r'', 0, regex = True)
stats = stats.rename(columns={'FantPos': 'Pos', 'Tm': 'Team', 'PPR': 'Points'})
stats.columns.values[7] = 'Pass Att'
stats.columns.values[8] = 'Pass Y'
stats.columns.values[9] = 'Pass TD'
stats.columns.values[11] = 'Rush Att'
stats.columns.values[12] = 'Rush Y'
stats.columns.values[14] = 'Rush TD'
stats.columns.values[17] = 'Rec Y'
stats.columns.values[19] = 'Rec TD'

In [7]:
stats = stats[~stats['Int'].isna()]
stats = stats.drop(columns = ['DKPt', 'FantPt', 'FDPt', 'VBD'])

In [8]:
conv_dict = {'Age': int,
             'G': int,
             'GS': int,
             'Cmp': int,
             'Pass Att': int,
             'Pass Y': int,
             'Pass TD': int,
             'Int': int,
             'Rush Att': int,
             'Rush Y': int,
             'Y/A': float,
             'Rush TD': int,
             'Tgt': int,
             'Rec': int,
             'Rec Y': int,
             'Y/R': float,
             'Rec TD': int,
             'Fmb': int,
             'FL': int,
             'TD': int,
             '2PM': int,
             '2PP': int,
             'Points': float,
             'PosRank': int,
             'OvRank': int
             }

In [9]:
stats = stats.astype(conv_dict)

In [10]:
stats['Pts/G'] = (stats['Points'] / stats['G']).round(1)
stats['Rec/Tgt'] = (stats['Rec'] / stats['Tgt'])
stats['Pts/Tgt'] = (stats['Points'] / stats['Tgt'])
stats['Touches'] = (stats['Rush Att'] + stats['Rec'])
stats['Player'] = stats['Player'].str.rstrip('*+')

In [11]:
stats

,Player,Team,Pos,Age,G,GS,Cmp,Pass Att,Pass Y,Pass TD,...,TD,2PM,2PP,Points,PosRank,OvRank,Pts/G,Rec/Tgt,Pts/Tgt,Touches
0,Jonathan Taylor,IND,RB,22,17,17,0,0,0,0,...,20,0,0,373.1,1,1,21.9,0.784314,7.315686,372
1,Cooper Kupp,LAR,WR,28,17,17,0,1,0,0,...,16,1,0,439.5,1,2,25.9,0.759162,2.301047,149
2,Deebo Samuel,SFO,WR,25,16,15,1,2,24,1,...,14,0,0,339.0,2,3,21.2,0.636364,2.801653,136
3,Josh Allen,BUF,QB,25,17,17,409,646,4407,36,...,6,2,1,402.6,1,4,23.7,NaN,inf,122
4,Austin Ekeler,LAC,RB,26,16,16,0,0,0,0,...,20,2,0,343.8,2,5,21.5,0.744681,3.657447,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,Travis Benjamin,SFO,0,32,10,0,0,0,0,0,...,0,0,0,-2.0,260,0,-0.2,0.000000,-0.400000,0
667,Trenton Cannon,2TM,RB,27,12,0,0,0,0,0,...,0,0,0,-1.6,179,0,-0.1,NaN,-inf,3
668,John Wolford,LAR,QB,26,3,0,1,4,5,0,...,0,0,0,-1.9,84,0,-0.6,NaN,-inf,2
669,Josh Rosen,ATL,QB,24,4,0,2,11,19,0,...,0,0,0,-3.2,85,0,-0.8,NaN,-inf,0


In [12]:
stats.to_csv('2021PlayerStats.csv')

In [13]:
def player_csv(year):
    
    url = url = "https://www.pro-football-reference.com/years/{}/fantasy.htm#".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
    headers = headers[1:]
    
    rows = soup.findAll('tr', class_ = lambda table_rows: table_rows != "thead")
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
                   for i in range(len(rows))]
    player_stats = player_stats[2:]
    
    stats = pd.DataFrame(player_stats, columns = headers)
    
    stats = stats.replace(r'', 0, regex = True) # Replace empty values with 0 & Rename Columns
    stats = stats.rename(columns={'FantPos': 'Pos', 'Tm': 'Team', 'PPR': 'Points'}) 
    stats.columns.values[7] = 'Pass Att'
    stats.columns.values[8] = 'Pass Y'
    stats.columns.values[9] = 'Pass TD'
    stats.columns.values[11] = 'Rush Att'
    stats.columns.values[12] = 'Rush Y'
    stats.columns.values[14] = 'Rush TD'
    stats.columns.values[17] = 'Rec Y'
    stats.columns.values[19] = 'Rec TD'
    
    stats = stats[~stats['Int'].isna()] #Remove all NULL values
    stats = stats.drop(columns = ['DKPt', 'FantPt', 'FDPt', 'VBD'])
    
    conv_dict = {'Age': int,
             'G': int,
             'GS': int,
             'Cmp': int,
             'Pass Att': int,
             'Pass Y': int,
             'Pass TD': int,
             'Int': int,
             'Rush Att': int,
             'Rush Y': int,
             'Y/A': float,
             'Rush TD': int,
             'Tgt': int,
             'Rec': int,
             'Rec Y': int,
             'Y/R': float,
             'Rec TD': int,
             'Fmb': int,
             'FL': int,
             'TD': int,
             '2PM': int,
             '2PP': int,
             'Standard': int,
             'PPR': float,
             'PosRank': int,
             'OvRank': int
             }
    stats = stats.astype(conv_dict) # Change data types of columns    
    
    stats['Pts/G'] = (stats['Points'] / stats['G']).round(1) # Create new columns
    stats['Year'] = year
    
    
    stats.to_csv('{}playerstats.csv'.format(year))

In [14]:
#player_csv(2020)
#player_csv(2019)
#player_csv(2018)
#player_csv(2017)
#player_csv(2016)

### Aggregate play-by-play data to find statistics for each player on a weekly basis

In [15]:
#YEAR = 2021

#data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
#                         'play_by_play_' + str(YEAR) + '.csv.gz?raw=True',
#                         compression='gzip', low_memory=False)

In [16]:
#data

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,NaN,NaN,NaN,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,-1.399805,NaN,NaN,NaN,NaN,NaN,0.491433,-49.143299
3,76,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
4,100,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50707,3930,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,CIN,home,LA,...,0,0,-1.242206,NaN,NaN,NaN,NaN,NaN,0.681295,-68.129545
50708,3951,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,NaN,NaN,NaN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50709,3968,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,CIN,home,LA,...,0,0,-0.569208,0.263600,4.354299,2.0,0.998861,0.998861,0.747601,25.239891
50710,3990,2021_22_LA_CIN,2022021301,CIN,LA,POST,22,LA,away,CIN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#for col_name in data.columns:
#    print(col_name)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [18]:
#passer = data.groupby(['passer', 'week'], as_index=False).agg({'passing_yards':'sum',
#                                                                'pass_touchdown':'sum',
#                                                                'interception':'sum',
#                                                                })
                                                    

In [19]:
#rusher = data.groupby(['rusher', 'fantasy_player_id', 'week'], as_index=False).agg({'rushing_yards':'sum',
#                                                                                    'rush_touchdown':'sum',
#                                                                                    'touchdown':'sum',
#                                                                                    'fumble':'sum'})
                                                                  
                                                                  
                                                                  

In [20]:
#receiver = data.groupby(['receiver', 'fantasy_player_id', 'week'], as_index=False).agg({'receiver_player_name':'count',
#                                                                                        'receiving_yards':'sum',
#                                                                                        })
                                                        
                                                        
                                                                  
                                                                  
                                                                  

In [21]:
#receiver

,receiver,fantasy_player_id,week,receiver_player_name,receiving_yards
0,A.Abdullah,00-0032104,1,2,15.0
1,A.Abdullah,00-0032104,2,1,0.0
2,A.Abdullah,00-0032104,4,1,2.0
3,A.Abdullah,00-0032104,8,5,35.0
4,A.Abdullah,00-0032104,9,5,30.0
...,...,...,...,...,...
4618,Z.Pascal,00-0033251,13,3,6.0
4619,Z.Pascal,00-0033251,15,2,23.0
4620,Z.Pascal,00-0033251,17,2,8.0
4621,Z.Pascal,00-0033251,18,2,7.0


## Read in game logs using CSV file from advancedsportsanalytics.com/nfl-raw-data

In [22]:
games = pd.read_csv('2021GameLogs.csv')

In [23]:
games['game_date'] = pd.to_datetime(games['game_date'])

In [24]:
games = games.drop(columns = ['game_id', 'rush_scrambles', 'designed_rush_att', 'comb_pass_rush_play', 'comb_pass_play', 'comb_rush_play',
                             'total_ret_td', 'pass_yds_bonus', 'rush_yds_bonus', 'rec_yds_bonus', 'Total_DKP', 'Off_DKP', 'Total_FDP',
                             'Off_FDP', 'Total_SDP', 'Off_SDP', 'pass_target_yds', 'pass_poor_throws', 'pass_blitzed', 'pass_hurried',
                             'rush_yds_before_contact', 'rush_yac', 'rec_air_yds', 'rec_yac', 'rec_drops', 'offense', 'off_pct', 'vis_team',
                             'player_id', 'home_team', 'OT', 'Temperature', 'Humidity', 'Wind_Speed', 'Vegas_Line', 'Vegas_Favorite', 
                              'Over_Under'])

In [26]:
games['week'] = ''

In [27]:
for ind, row in games.iterrows():
    if row['game_date'] >= datetime.strptime('2021-09-09', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-09-13', '%Y-%m-%d'):
        games.at[ind, 'week'] = 1
    elif row['game_date'] >= datetime.strptime('2021-09-16', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-09-20', '%Y-%m-%d'):
        games.at[ind, 'week'] = 2
    elif row['game_date'] >= datetime.strptime('2021-09-23', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-09-27', '%Y-%m-%d'):
        games.at[ind, 'week'] = 3
    elif row['game_date'] >= datetime.strptime('2021-09-30', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-10-04', '%Y-%m-%d'):
        games.at[ind, 'week'] = 4
    elif row['game_date'] >= datetime.strptime('2021-10-07', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-10-11', '%Y-%m-%d'):
        games.at[ind, 'week'] = 5
    elif row['game_date'] >= datetime.strptime('2021-10-14', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-10-18', '%Y-%m-%d'):
        games.at[ind, 'week'] = 6
    elif row['game_date'] >= datetime.strptime('2021-10-21', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-10-25', '%Y-%m-%d'):
        games.at[ind, 'week'] = 7
    elif row['game_date'] >= datetime.strptime('2021-10-28', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-11-01', '%Y-%m-%d'):
        games.at[ind, 'week'] = 8
    elif row['game_date'] >= datetime.strptime('2021-11-04', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-11-08', '%Y-%m-%d'):
        games.at[ind, 'week'] = 9
    elif row['game_date'] >= datetime.strptime('2021-11-11', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-11-15', '%Y-%m-%d'):
        games.at[ind, 'week'] = 10
    elif row['game_date'] >= datetime.strptime('2021-11-18', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-11-22', '%Y-%m-%d'):
        games.at[ind, 'week'] = 11
    elif row['game_date'] >= datetime.strptime('2021-11-25', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-11-29', '%Y-%m-%d'):
        games.at[ind, 'week'] = 12
    elif row['game_date'] >= datetime.strptime('2021-12-02', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-12-06', '%Y-%m-%d'):
        games.at[ind, 'week'] = 13
    elif row['game_date'] >= datetime.strptime('2021-12-09', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-12-13', '%Y-%m-%d'):
        games.at[ind, 'week'] = 14
    elif row['game_date'] >= datetime.strptime('2021-12-16', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-12-21', '%Y-%m-%d'):
        games.at[ind, 'week'] = 15
    elif row['game_date'] >= datetime.strptime('2021-12-23', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2021-12-27', '%Y-%m-%d'):
        games.at[ind, 'week'] = 16
    elif row['game_date'] >= datetime.strptime('2022-01-02', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2022-01-03', '%Y-%m-%d'):
        games.at[ind, 'week'] = 17
    elif row['game_date'] >= datetime.strptime('2022-01-08', '%Y-%m-%d') and row['game_date'] <= datetime.strptime('2022-01-09', '%Y-%m-%d'):
        games.at[ind, 'week'] = 18

In [28]:
games['PPR Pts'] = ((games['pass_td'] * 4) + (games['pass_yds'] / 25) + (games['two_point_conv'] * 2) + 
                   (games['rush_td'] * 6) + (games['rush_yds'] / 10) + (games['rec_td'] * 6) + (games['rec_yds'] / 10) + 
                   (games['rec']) + (games['pass_int'] * -2) + (games['fumbles_lost'] * -2)).round(1)

In [29]:
weekly = games[['player', 'team', 'pos', 'week', 'PPR Pts']]
weekly = weekly.pivot(index=['player', 'team', 'pos'], columns='week')
weekly.columns = weekly.columns.droplevel(0)

In [30]:
weekly['Consistency'] = weekly[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]].std(axis=1)

In [31]:
weekly.reset_index(inplace=True)
weekly = weekly.rename(columns = {'player':'Player', 'team':'Team', 'pos':'Pos'})

In [32]:
total = stats.merge(weekly, how = 'inner', on = ['Player', 'Team', 'Pos'])

In [34]:
total

,Player,Team,Pos,Age,G,GS,Cmp,Pass Att,Pass Y,Pass TD,...,10,11,12,13,14,15,16,17,18,Consistency
0,Jonathan Taylor,IND,RB,22,17,17,0,0,0,0,...,24.6,53.4,19.7,24.3,NaN,23.0,10.8,18.4,12.5,11.170951
1,Cooper Kupp,LAR,WR,28,17,17,0,1,0,0,...,23.2,NaN,18.6,26.9,31.3,34.7,21.3,21.5,26.6,7.468360
2,Deebo Samuel,SFO,WR,25,16,15,1,2,24,1,...,30.3,16.4,20.8,NaN,12.9,18.9,28.1,17.2,29.0,7.763408
3,Josh Allen,BUF,QB,25,17,17,409,646,4407,36,...,20.9,16.2,26.7,11.7,35.2,20.8,31.0,20.9,23.9,8.394186
4,Austin Ekeler,LAC,RB,26,16,16,0,0,0,0,...,14.9,41.5,21.9,17.4,16.4,18.2,NaN,19.8,28.9,8.586802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,Malik Taylor,GNB,WR,26,10,0,0,0,0,0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1.202082
551,Logan Woodside,TEN,QB,26,5,0,0,0,0,0,...,NaN,NaN,NaN,NaN,-0.3,NaN,NaN,NaN,NaN,0.100000
552,John Wolford,LAR,QB,26,3,0,1,4,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.626346
553,Josh Rosen,ATL,QB,24,4,0,2,11,19,0,...,-1.4,-1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.945163
